In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.model_selection import train_test_split

import collections
import torch
import torch.nn as nn
from torch import optim
import random
import time

import warnings;
warnings.filterwarnings('ignore');

# import process_tweet
# import importlib
# importlib.reload(process_tweet)

import numpy as np
import pandas as pd
from pprint import pprint

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.model_selection import train_test_split

#create spell checker/word splitter
def create_symspell(max_edit_distance, prefix_length, freq_file_path):
    # create object
    sym_spell = SymSpell(max_edit_distance, prefix_length)
    
    # create dictionary using corpus.txt
    if not sym_spell.create_dictionary(freq_file_path):
        print("Corpus file not found")
        return None
    return sym_spell

def is_valid_token(w):
    special = ['<url>','<number>', '<user>']
    return w.isalpha() or w in special

def process_tweet(tweet, tknzr, sym_spell=None, advanced=False):
    st_1 = []
    for w in tknzr.tokenize(tweet):
        #remove retweet annotation if present:
        if w == 'RT':
            if advanced:
                st_1.append('rt')
        elif w[0] == '@':
            if advanced:
                st_1.append('<user>')
        #remove hashtag symbol
        elif w[0] == '#':
            st_1.append(w[1:])
        #replace link with LINK keyword
        elif w[:4] == 'http':
            st_1.append('<url>')
        elif w.isnumeric():
            if advanced:
                st_1.append('<number>')
        else:
            st_1.append(w)
    
    st_2 = []
    
    #remove stop words and punctuation, make everything lowercase
    if sym_spell != None:
        st_2 = [sym_spell.word_segmentation(w.lower()).corrected_string 
                for w in st_1 if w.isalpha() and not w.lower() in stop_words]
    elif advanced:
        st_2 = [w.lower() for w in st_1 if is_valid_token(w) and 
                    not w.lower() in stop_words]
    else:
        st_2 = [w.lower() for w in st_1 if w.isalpha() and
                not w.lower() in stop_words]
    
    #lemmatization (converts all words to root form for standardization)
    lem = WordNetLemmatizer()
    st_3 = list(map(lambda x: lem.lemmatize(x, pos='v'), st_2))
    
    #now do word segmentation/spell check
    return ' '.join(st_3)

[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ashwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
df = pd.read_csv('data/labeled_prelim.csv').dropna().astype(
        {'Relevancy':np.int32, 'Urgency':np.int32}).reset_index(drop=True)
df.pop('Id')

df2 = pd.read_csv('data/mturk_results_0-2000_processed.csv')
#df = pd.concat([df, df2], ignore_index=True)
df = df2

print(df.shape)
df.head()

(1741, 3)


,Text,Urgency,Relevancy
0,"your rescue boats, vehicles, volunteer craft ...",0,0
1,"!! According to #PBS, #Houston convention cen...",0,1
2,"""100,000 homes have been damaged by #Harvey an...",0,2
3,"""Do you need us to get a boat?"" Things I never...",0,0
4,"""Flood of epic proportions."" #HurricaneHarvey ...",0,3


In [12]:
#sym_spell = create_symspell(2,7,'data/frequency_dictionary_en_82_765.txt')
tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)
df['Text'] = df['Text'].map(lambda x: process_tweet(x, tknzr, None, True))
df.head()

,Text,Urgency,Relevancy
0,rescue boat vehicles volunteer craft need txwx...,0,0
1,accord pbs houston convention center need whee...,0,1
2,home damage harvey financial toll begin <url>,0,2
3,need us get boat things never think ask friend...,0,0
4,flood epic proportion hurricaneharvey inundate...,0,3


In [5]:
process_tweet('@user #hello world', tknzr, None,True)

'<user> hello world'

In [6]:
#list of embeddings
vec_length = 50
embeddings = np.zeros((1193514, vec_length))

#two-way map, index->word and word->index
glove = {}

index = 0
with open('data/glove.twitter.27B/glove.twitter.27B.%dd.txt' % vec_length) as f:
    for l in f:
        line = []
        try:
            line = l.split()
            if len(line) != vec_length+1:
                print('empty line')
                continue
            
            word = line[0]
            embeddings[index] = np.array(line[1:]).astype(np.float)
            glove[index] = word
            glove[word] = index
            index += 1
        except:
            print(line)
            print(index)
            break

empty line


In [23]:
#only handles binary classification for now
def tweets_to_df(df, labels, embeddings, glove):
    
    weights = []
    index_omit = []
    index = -1
    tweets = df['Text']
    
    #a column for each entry in the embedding vector
    for i in range(vec_length+1):
        weights.append([])
    
    for i in range(len(tweets)):
        index += 1
        cur_embed = []
        cur_tweet = tweets[i]
        cur_label = labels[i]
        for i in cur_tweet.split():
            if i in glove:
                cur_embed.append(embeddings[glove[i]])
        
        if len(cur_embed) == 0:
            #make sure we drop this row from the input dataframe
            index_omit.append(index)
            continue
        
        x = np.asarray(np.mean(cur_embed, axis=0))
        
        for j in range(vec_length):
            weights[j].append(x[j])
        weights[vec_length].append(0 if cur_label == 0 else 1)
        #weights[vec_length].append(cur_label)
        
    df_pruned = df.drop(index_omit)
    
    #convert to dataframe
    cols = {}
    for i in range(vec_length):
       cols['v' + str(i)] = weights[i]
    
    cols['class'] = weights[vec_length]
    
    df2 = pd.DataFrame(data=cols)
    return df2

In [26]:
dfv = tweets_to_df(df, df['Relevancy'], embeddings, glove)
labels = dfv.pop('class')
dfv.head()

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49
0,0.018780,0.019162,-0.204499,-0.702597,-0.041929,-0.200875,0.487510,-0.261408,0.475035,-0.831866,...,-0.097450,0.046975,0.353386,0.013975,0.025537,0.338198,0.196569,-0.062159,0.185096,0.603920
1,0.115822,0.151914,-0.252038,-0.271220,0.005090,-0.236383,0.261833,-0.037649,0.440095,-0.387786,...,-0.351785,0.037818,0.001821,-0.001865,0.229696,0.023982,0.208820,-0.071991,-0.145986,-0.214229
2,0.142689,0.119957,-0.359924,-0.618667,0.152206,0.008059,0.015033,-0.316391,-0.005230,-0.621019,...,-0.572997,0.111872,0.359161,0.112237,0.236217,-0.093718,-0.026002,-0.046747,-0.053574,0.614299
3,0.129950,0.395938,-0.180748,-0.024789,-0.156947,-0.214214,0.889963,-0.050893,0.223831,0.224263,...,-0.492443,0.448336,0.199528,-0.159725,0.325608,-0.147479,0.252455,0.166972,-0.021204,0.269543
4,0.223490,0.356330,-0.052827,-0.479262,0.179755,-0.060940,0.522521,-0.223807,0.280253,-0.206572,...,-0.523289,-0.134802,0.155163,0.086522,-0.065962,0.267316,-0.133432,0.090854,0.073698,0.391386


In [27]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import * 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import *
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

def average(x):
    return sum(x)/len(x)

def get_stats(model, X, y, cv, verbose=False):
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
    auc = []
        
    cv_results = cross_validate(model, X, y, scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], 
                                cv=cv, return_train_score=False)
    
    if verbose:
        print(cv_results)
    
    #now return the data
    return cv_results

In [28]:
models = {'Naive Bayes': GaussianNB(),
          'Voting': VotingClassifier(estimators=[('mlp', MLPClassifier()),
                                            ('ada', AdaBoostClassifier()),
                                            ('nb', GaussianNB())], voting='soft'),
          'Perceptron': MLPClassifier(),
          'AdaBoost': AdaBoostClassifier()}

vals = []
metric = []
model_name = []

f1 = []
precision = []
recall = []
accuracy = []
auc = []
method = ['Naive Bayes', 'Voting', 'MLP', 'AdaBoost']

cv = 10
for k,v in models.items():
    stats = get_stats(v, dfv, labels, cv)
    accuracy_avg = np.average(stats['test_accuracy'])
    accuracy_std = np.std(stats['test_accuracy'])
    precision_avg = np.average(stats['test_precision'])
    precision_std = np.std(stats['test_precision'])
    recall_avg = np.average(stats['test_recall'])
    recall_std = np.std(stats['test_recall'])
    f1_avg = np.average(stats['test_f1'])
    f1_std = np.std(stats['test_f1'])
    auc_avg = np.average(stats['test_roc_auc'])
    
    f1.append('%.2f ± %.2f' % (f1_avg, f1_std))
    precision.append('%.2f ± %.2f' % (precision_avg, precision_std))
    recall.append('%.2f ± %.2f' % (recall_avg, recall_std))
    accuracy.append('%.2f ± %.2f' % (accuracy_avg, accuracy_std))
    auc.append('%.2f' % auc_avg)

df_view = pd.DataFrame(data={'Method': method, 'f1': f1, 
                             'precision':precision, 'recall':recall,
                             'accuracy':accuracy, 'auc':auc})
df_view

,Method,f1,precision,recall,accuracy,auc
0,Naive Bayes,0.66 ± 0.04,0.57 ± 0.05,0.78 ± 0.06,0.62 ± 0.05,0.70
1,Voting,0.67 ± 0.03,0.60 ± 0.05,0.75 ± 0.05,0.66 ± 0.05,0.75
2,MLP,0.66 ± 0.03,0.66 ± 0.05,0.66 ± 0.04,0.68 ± 0.04,0.74
3,AdaBoost,0.64 ± 0.06,0.64 ± 0.07,0.64 ± 0.08,0.66 ± 0.06,0.72


In [11]:
#now generate a dataset for MTurk from the subset of 300k tweets
df_subset = pd.read_csv('data/shuffled_subset_all.csv', lineterminator='\n').drop(['Unnamed: 0'], axis=1).loc[1500:,:]
print(df_subset.shape)
df_subset.head()

(364956, 1)


,text
1500,"Ok. Houston, stay home. Stay out of the attic...."
1501,Trump killed flood safety rules 10 days before...
1502,@funder @KariannHart Take that fucking wall do...
1503,water vapor satellite image shows extensive dr...
1504,Alcohol helps cause traffic accidents.\nSmokin...


In [12]:
#a crude way to filter out some donation tweets
prune_indices = df_subset[df_subset['text'].str.contains('donat')].index.values.tolist()
df_subset = df_subset.drop(prune_indices)
print(df_subset.shape)
df_subset.head()

(320472, 1)


,text
1500,"Ok. Houston, stay home. Stay out of the attic...."
1501,Trump killed flood safety rules 10 days before...
1502,@funder @KariannHart Take that fucking wall do...
1503,water vapor satellite image shows extensive dr...
1504,Alcohol helps cause traffic accidents.\nSmokin...


In [13]:
from langdetect import detect

#now get rid of non-English stuff
drop_indices =[]
for index, row in enumerate(df_subset.itertuples()):
    
    if index >= 20000:
        break
    elif index % 10000 == 0:
        print(index)
    
    lang = detect(row.text)
    if lang !='en' and lang != 'nl':
        drop_indices.append(index)

0
10000


In [13]:
#now prune the tweets that are not English
df_english = df_subset.drop(df_subset.index[drop_indices])
df_english.shape

(320373, 1)

In [17]:
df_english_small = df_english.iloc[:20000,:]
print(df_english_small.shape)
df_english_small.head()

(20000, 1)


,text
1500,"Ok. Houston, stay home. Stay out of the attic...."
1501,Trump killed flood safety rules 10 days before...
1502,@funder @KariannHart Take that fucking wall do...
1503,water vapor satellite image shows extensive dr...
1504,Alcohol helps cause traffic accidents.\nSmokin...


In [20]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
df_noemoji = pd.DataFrame(data={'text' : df_english_small['text'].map(deEmojify)})
print(df_noemoji.shape)
df_noemoji.head()

(20000, 1)


,text
1500,"Ok. Houston, stay home. Stay out of the attic...."
1501,Trump killed flood safety rules 10 days before...
1502,@funder @KariannHart Take that fucking wall do...
1503,water vapor satellite image shows extensive dr...
1504,Alcohol helps cause traffic accidents.\nSmokin...


In [21]:
#now preprocess the text
df_test = df_noemoji.copy().reset_index(drop=True)
df_test['Text'] = df_test['text'].map(lambda x: process_tweet(x, tknzr, None, True))
df_test.pop('text')

print(df_test.shape)
df_test.head()

(20000, 1)


,Text
0,ok houston stay home stay attic call <number> ...
1,trump kill flood safety rule <number> days har...
2,<user> <user> take fuck wall <user> hurricaneh...
3,water vapor satellite image show extensive dry...
4,alcohol help cause traffic accidents smoke hel...


In [22]:
#now convert to word embeddings
df_testv = tweets_to_df(df_test, [0]*df_test.shape[0], embeddings, glove).reset_index(drop=True)
df_testv.pop('class')
df_testv.head()

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49
0,-0.043563,0.168890,0.132577,-0.318665,-0.148289,0.082531,0.732835,0.150738,0.097140,-0.027730,...,-0.726723,0.383673,0.364157,-0.003794,0.233437,-0.222859,0.229726,-0.269067,0.002645,0.182357
1,0.198020,0.111794,-0.194317,-0.372960,-0.037570,-0.161805,0.327506,0.018590,0.199540,-0.131544,...,-0.539943,-0.106992,0.183136,-0.061690,0.293251,-0.094791,-0.082968,-0.185012,-0.121363,0.213161
2,0.581150,0.288926,0.154654,-0.169325,0.123784,0.187431,0.353618,0.311923,0.172137,0.650069,...,-1.555811,-0.098209,0.217706,0.436886,0.221757,-0.083573,-0.025337,0.116973,-0.093931,-0.081701
3,-0.213230,-0.330857,0.081841,-0.154605,-0.170297,0.227414,0.515372,-0.379785,0.105189,-0.330809,...,-0.582300,-0.138565,-0.083400,-0.111499,0.082342,0.159363,0.187800,-0.086185,0.011892,0.233976
4,0.094960,0.266487,-0.848661,-0.214325,0.030654,0.126114,0.402066,-0.261091,0.113748,0.007945,...,-0.373645,-0.058034,0.059335,-0.090298,0.071710,-0.197974,0.188452,0.468361,-0.197567,0.282665


In [24]:
#now run the model
mlp = MLPClassifier()
mlp.fit(dfv, labels)
pred = mlp.predict(df_testv)

In [25]:
#now run until we have 5000 positives and 5000 negatives
pos = 0
neg = 0
indices = []
success = False

for index in range(len(pred)):
    p = pred[index]
    if pos == 2400 and neg == 1600:
        success = True
        print('looked through index ' + str(index))
        break
    elif p == 0:
        if neg < 1600:
            indices.append(index)
            neg += 1
    else:
        if pos < 2400:
            indices.append(index)
            pos += 1

print('successful' if success else 'failure')

looked through index 10783
successful


In [26]:
#now make the final dataframe containing 10000 tweets that are relevant and not relevant
df_mturk = df_noemoji.reset_index(drop=True).loc[indices].reset_index(drop=True)
df_mturk

,text
0,"Ok. Houston, stay home. Stay out of the attic...."
1,Trump killed flood safety rules 10 days before...
2,@funder @KariannHart Take that fucking wall do...
3,water vapor satellite image shows extensive dr...
4,Alcohol helps cause traffic accidents.\nSmokin...
5,@TXInstruments @tenethealth @jcpenney can we k...
6,"How horrible, look how high the water is on th..."
7,"Remember those we lost, those who helped us co..."
8,How @SpringISD is helping its community recove...
9,For over 100 yrs @VOATexas has served those in...


In [27]:
df_mturk.to_csv('data/mturk_final4000.csv', index=False)